In [ ]:
import os
import plotly
import quaternion
import numpy as np
import scipy.optimize as opt
import plotly.graph_objects as go

from tqdm import tqdm
from IPython.display import display, HTML

plotly.offline.init_notebook_mode()
display(HTML('<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'))
np.seterr(all='ignore')

In [ ]:
def E_func(E, M, e):
        return E - e*np.sin(E) - M
def E_prime(E, M, e):
    return 1 - e*np.cos(E)

def eccentric_anomaly(M, e):
    E = opt.newton(E_func, M+e, fprime=E_prime, args=(M,e),tol=1e-10,)
    return E

def mean_to_true_anomaly(M, e):
    E = eccentric_anomaly(M, e)
    f = 2*np.arctan2(np.tan(E/2)*np.sqrt(1+e), np.sqrt(1-e))
    if f > np.pi:
        f -= 2*np.pi
    if f < -np.pi:
        f += 2*np.pi
    return f
def disp_state_vec(state):
    print(f"R:\t [{state[0]}, {state[1]}, {state[2]}] m")
    print(f"V:\t [{state[3]}, {state[4]}, {state[5]}] m/s\n")

def disp_kep_elem(kep):
    print(f"Semimajor Axis (a):\t\t {kep[0]} m")
    print(f"Eccentricity (e):\t\t {kep[1]}")
    print(f"Inclination (i):\t\t {np.degrees(kep[2])}º")
    print(f"RAAN (Ω):\t\t\t {np.degrees(kep[3])}º")
    print(f"Argument of Periapse (𝜔):\t {np.degrees(kep[4])}º")
    print(f"True Anomaly (f):\t\t {np.degrees(kep[5])}º\n")

def state_vec_to_keplerian(state, mu):
    a = semimajor_axis(state, mu)
    e = np.linalg.norm(eccentricity(state, mu))
    i = inclination(state)
    raan = RAAN(state)
    omega = argument_of_periapse(state, mu)
    f = true_anomaly(state, mu)
    return np.array([a, e, i, raan, omega, f])

def keplerian_to_state_vec(kep, mu):
    pos = position(kep, mu)
    vel = velocity(kep, mu)

    state_vec = np.array([pos[0], pos[1], pos[2], vel[0], vel[1], vel[2]])
    disp_state_vec(state_vec)
    input = kepDeg2Rad(kep)
    check = state_vec_to_keplerian(state_vec, mu)
    if np.allclose(input, check, rtol=1e-04, atol=1e-05):
        return state_vec
    else:
        print("WARNING: Keplerian elements do not match")
        disp_kep_elem(input)
        disp_kep_elem(check)
        return state_vec

def semimajor_axis(state, mu):
    r = np.linalg.norm(state[0:3])
    v = np.linalg.norm(state[3:6])
    if r == 0:
        return 0
    a = 1/(2/r - v**2/mu)
    return a

def eccentricity(state, mu):
    r = state[0:3]
    v = state[3:6]
    r_mag = np.sqrt(np.dot(r, r))
    v_mag = np.sqrt(np.dot(v, v))
    if r_mag == 0:
        return 0
    e = (v_mag**2/mu - 1/r_mag)*r - 1/mu*np.dot(r, v)*v
    return e

def inclination(state):
    r = state[0:3]
    v = state[3:6]
    h = np.cross(r, v)
    h_mag = np.sqrt(np.dot(h, h))
    if h_mag == 0:
        return 0
    incl = np.arccos(np.dot(h/h_mag, np.array([0, 0, 1])))
    return incl

def node_vector(state):
    r = state[0:3]
    v = state[3:6]
    h = np.cross(r, v)
    n = np.cross(np.array([0, 0, 1]), h)
    return n

def RAAN(state):
    node = node_vector(state)
    node_mag = np.sqrt(np.dot(node, node))
    if node_mag == 0:
        return 0
    omega = np.arccos(np.dot(node, np.array([1, 0, 0])) / node_mag)
    if np.dot(node, np.array([0, 1, 0])) < 0:
        omega = 2*np.pi - omega
    return omega

def argument_of_periapse(state, mu):
    node = node_vector(state)
    node_mag = np.sqrt(np.dot(node, node))
    e = eccentricity(state, mu)
    e_mag = np.sqrt(np.dot(e, e))
    if node_mag == 0 or e_mag == 0:
        return 0
    omega = np.arccos(np.dot(node, e)/(node_mag*e_mag))
    if (np.dot(e, np.array([0, 0, 1])) < 0):
        omega = 2*np.pi - omega
    return omega

def true_anomaly(state, mu):
    r = state[0:3]
    e = eccentricity(state, mu)
    r_mag = np.sqrt(np.dot(r, r))
    e_mag = np.sqrt(np.dot(e, e))
    if r_mag == 0 or e_mag == 0:
        return 0
    if np.isclose(np.dot(e, r)/(e_mag*r_mag), 1):
        return 0
    f = np.arccos(np.dot(e, r)/(e_mag*r_mag))
    while f >= 2*np.pi:
        f -= 2*np.pi
    while f <= -2*np.pi:
        f += 2*np.pi
    return f

def angular_momentum_from_OE(a, e, mu):
    if e >=0 and e < 1:
        return np.sqrt(mu*a*(1-e**2))
    elif e < 0:
        return np.sqrt(mu*a*(e**2-1))

def position(kep, mu):
    a = kep[0]
    e = kep[1]
    i = np.radians(kep[2])
    raan = np.radians(kep[3])
    omega = np.radians(kep[4])
    f = np.radians(kep[5])

    r = a*(1-e**2)/(1+e*np.cos(f))
    theta = omega + f
    pos = r * np.array([np.cos(theta)*np.cos(raan) - np.cos(i)*np.sin(raan)*np.sin(theta),
                        np.cos(theta)*np.sin(raan) + np.cos(i)*np.cos(raan)*np.sin(theta),
                        np.sin(i)*np.sin(theta)])
    return pos

def velocity(kep, mu):
    a = kep[0]
    e = kep[1]
    inc = np.radians(kep[2])
    raan = np.radians(kep[3])
    omega = np.radians(kep[4])
    f = np.radians(kep[5])

    theta = omega + f

    h = angular_momentum_from_OE(a, e, mu)
    theta = omega + f
    vel_vec = mu/h*np.array([-(np.cos(raan)*(np.sin(theta) + e*np.sin(omega)) + np.sin(raan)*(np.cos(theta)+ e*np.cos(omega))*np.cos(inc)),
                             -(np.sin(raan)*(np.sin(theta) + e*np.sin(omega)) - np.cos(raan)*(np.cos(theta) + e*np.cos(omega))*np.cos(inc)),
                             (np.cos(theta) + e*np.cos(omega))*np.sin(inc)])              
    return vel_vec

def kepRad2Deg(kep):
    return np.array([kep[0], kep[1], np.degrees(kep[2]), np.degrees(kep[3]), np.degrees(kep[4]), np.degrees(kep[5])])

def kepDeg2Rad(kep):
    return np.array([kep[0], kep[1], np.radians(kep[2]), np.radians(kep[3]), np.radians(kep[4]), np.radians(kep[5])]) 

def unit(vec):
    return vec/np.linalg.norm(vec)  

def get_error_quat(q1, q2):
    if np.dot(quaternion.as_float_array(q1), quaternion.as_float_array(q2)) < 0:
        q1 = -q1
    return q1 * q2.inverse()

def get_error_quats(q1s, q2s):
    q_errs = np.zeros((len(q1s),4))
    for i in range(len(q1s)):
        q1          = quaternion.from_float_array(q1s[i])
        q2          = quaternion.from_float_array(q2s[i])
        dq          = get_error_quat(q1, q2)
        q_err       = quaternion.as_float_array(dq)
        q_errs[i]   = q_err
    return q_errs

In [ ]:
r_moon = 1737.1e3
mu_moon = 4.9048695e12

In [ ]:
working_dir     = os.getcwd()
save_file_name  = "sim_att.npy"
save_file       = os.path.join(working_dir, save_file_name)
sim_data = np.load(save_file, allow_pickle=True)

times                       = sim_data[0]
state_hist                  = sim_data[1]
enu_state_hist              = sim_data[2]
q_inertial_to_target_hist   = sim_data[3]
LVCS_state_hist             = sim_data[4]
thrust_dir_hist             = sim_data[5]
idx_events                  = sim_data[6]
guid_params                 = sim_data[7]

csm_state_vec   = guid_params[0]
man_input       = guid_params[2]
auto_input      = guid_params[3]

veh_params      = auto_input[0]

R_D, Y_D, R_D_dot, Y_D_dot, Z_D_dot = man_input[1:]
csm_orbital_elements                = state_vec_to_keplerian(csm_state_vec, mu_moon)
mass_dry                            = veh_params[2]

event_names = ["Lift-Off", "Start Azimuth Align", "Start Orbit Insertion", "Cut-Off"]


terminal_state  = state_hist[-1]
kep_final       = state_vec_to_keplerian(terminal_state, mu_moon)
print("CSM Initial Keplerian Elements:")
disp_kep_elem(csm_orbital_elements)
print("LM Final Keplerian Elements:")
disp_kep_elem(kep_final)

In [ ]:
def draw_planet(radius, colorscale):
    # Polar coordinates
    theta   = np.linspace(0, 2.*np.pi, 40)
    phi     = np.linspace(0, np.pi, 40)

    # Convert to cartesian
    x       = radius * np.outer(np.cos(theta), np.sin(phi))
    y       = radius * np.outer(np.sin(theta), np.sin(phi))
    z       = radius * np.outer(np.ones(np.size(theta)), np.cos(phi))

    # Create the planet object
    planet  = go.Surface(x=x, y=y, z=z, opacity=1.0, colorscale=colorscale, showscale=False)
    return planet

def draw_target_axes(pos, orient, scale):
    rot_targ    = quaternion.as_rotation_matrix(quaternion.from_float_array(orient))

    # Plot the target axes
    x_target_x_comps    = [pos[0], pos[0] + rot_targ[0,0]*scale]
    x_target_y_comps    = [pos[1], pos[1] + rot_targ[0,1]*scale]
    x_target_z_comps    = [pos[2], pos[2] + rot_targ[0,2]*scale]
    x_target_obj        = go.Scatter3d(x=x_target_x_comps,
                                       y=x_target_y_comps,
                                       z=x_target_z_comps,
                                       mode="lines", 
                                       line=dict(width=3, color="purple"),
                                       showlegend=False)
    
    y_target_x_comps    = [pos[0], pos[0] + rot_targ[1,0]*scale]
    y_target_y_comps    = [pos[1], pos[1] + rot_targ[1,1]*scale]
    y_target_z_comps    = [pos[2], pos[2] + rot_targ[1,2]*scale]
    y_target_obj        = go.Scatter3d(x=y_target_x_comps,
                                       y=y_target_y_comps,
                                       z=y_target_z_comps,
                                       mode="lines", 
                                       line=dict(width=3, color="purple"),
                                       showlegend=False)
    
    z_target_x_comps    = [pos[0], pos[0] + rot_targ[2,0]*scale]
    z_target_y_comps    = [pos[1], pos[1] + rot_targ[2,1]*scale]
    z_target_z_comps    = [pos[2], pos[2] + rot_targ[2,2]*scale]
    z_target_obj        = go.Scatter3d(x=z_target_x_comps,
                                       y=z_target_y_comps,
                                       z=z_target_z_comps,
                                       mode="lines", 
                                       line=dict(width=3, color="purple"),
                                       showlegend=False)
    
    # Plot cones at the ends of the target axes
    x_target_cone_x     = [pos[0] + rot_targ[0,0]*scale]
    x_target_cone_y     = [pos[1] + rot_targ[0,1]*scale]
    x_target_cone_z     = [pos[2] + rot_targ[0,2]*scale]
    x_target_cone_u     = [rot_targ[0,0]*scale*.3]
    x_target_cone_v     = [rot_targ[0,1]*scale*.3]
    x_target_cone_w     = [rot_targ[0,2]*scale*.3]
    x_targ_cone         = go.Cone(x=x_target_cone_x,
                                  y=x_target_cone_y,
                                  z=x_target_cone_z,
                                  u=x_target_cone_u,
                                  v=x_target_cone_v,
                                  w=x_target_cone_w,
                                  anchor = "tail",
                                  hoverinfo =  None,
                                  colorscale = [[0, "red"], [1, "red"]],
                                  showscale = False, showlegend=False)
    
    y_target_cone_x     = [pos[0] + rot_targ[1,0]*scale]
    y_target_cone_y     = [pos[1] + rot_targ[1,1]*scale]
    y_target_cone_z     = [pos[2] + rot_targ[1,2]*scale]
    y_target_cone_u     = [rot_targ[1,0]*scale*.3]
    y_target_cone_v     = [rot_targ[1,1]*scale*.3]
    y_target_cone_w     = [rot_targ[1,2]*scale*.3]
    y_targ_cone         = go.Cone(x=y_target_cone_x,
                                  y=y_target_cone_y,
                                  z=y_target_cone_z,
                                  u=y_target_cone_u,
                                  v=y_target_cone_v,
                                  w=y_target_cone_w,
                                  anchor = "tail",
                                  hoverinfo =  None,
                                  colorscale = [[0, "green"], [1, "green"]],
                                  showscale = False, showlegend=False)
                          
    z_targ_cone_x       = [pos[0] + rot_targ[2,0]*scale]
    z_targ_cone_y       = [pos[1] + rot_targ[2,1]*scale]
    z_targ_cone_z       = [pos[2] + rot_targ[2,2]*scale]
    z_targ_cone_u       = [rot_targ[2,0]*scale*.3]
    z_targ_cone_v       = [rot_targ[2,1]*scale*.3]
    z_targ_cone_w       = [rot_targ[2,2]*scale*.3]
    z_targ_cone         = go.Cone(x=z_targ_cone_x,
                                  y=z_targ_cone_y,
                                  z=z_targ_cone_z,
                                  u=z_targ_cone_u,
                                  v=z_targ_cone_v,
                                  w=z_targ_cone_w,
                                  anchor = "tail",
                                  hoverinfo =  None,
                                  colorscale = [[0, "blue"], [1, "blue"]],
                                  showscale = False, showlegend=False)
    
    return [x_target_obj, y_target_obj, z_target_obj, x_targ_cone, y_targ_cone, z_targ_cone]

def draw_spacecraft_axes(pos, orient, scale):
    # Convert the quaternion to a rotation matrix to get the axes
    rot         = quaternion.as_rotation_matrix(quaternion.from_float_array(orient))

    # Create the data for the plot
    # Plot the spacecraft axes
    x_x_comps   = [pos[0], pos[0] + rot[0,0]*scale]
    x_y_comps   = [pos[1], pos[1] + rot[0,1]*scale]
    x_z_comps   = [pos[2], pos[2] + rot[0,2]*scale]
    x_axis_obj  = go.Scatter3d(x=x_x_comps,
                               y=x_y_comps,
                               z=x_z_comps,
                               mode="lines", 
                               line=dict(width=3, color="red"), showlegend=False)
    
    y_x_comps   = [pos[0], pos[0] + rot[1,0]*scale]
    y_y_comps   = [pos[1], pos[1] + rot[1,1]*scale]
    y_z_comps   = [pos[2], pos[2] + rot[1,2]*scale]
    y_axis_obj  = go.Scatter3d(x=y_x_comps,
                               y=y_y_comps,
                               z=y_z_comps,
                               mode="lines", 
                               line=dict(width=3, color="green"), showlegend=False)
    
    z_x_comps   = [pos[0], pos[0] + rot[2,0]*scale]
    z_y_comps   = [pos[1], pos[1] + rot[2,1]*scale]
    z_z_comps   = [pos[2], pos[2] + rot[2,2]*scale]
    z_axis_obj  = go.Scatter3d(x=z_x_comps,
                               y=z_y_comps,
                               z=z_z_comps,
                               mode="lines", 
                               line=dict(width=3, color="blue"), showlegend=False)
    return [x_axis_obj, y_axis_obj, z_axis_obj]

def draw_traj(posits, idx, colorscale):
    spacecraft_traj  = go.Scatter3d(x=posits[0:idx,0],
                                    y=posits[0:idx,1],
                                    z=posits[0:idx,2],
                                    mode="lines",
                                    line=dict(width=5, colorscale=colorscale, 
                                              color=np.arange(0, idx, 1), 
                                              showscale=False), showlegend=False)
    return [spacecraft_traj]

def states_for_plot(sat_pos, sat_orient, sat_quat_targ):
    sat_state_plot = np.zeros((len(sat_pos), 11))
    sat_state_plot[:,0:3]   = sat_pos
    sat_state_plot[:,3:7]   = sat_orient
    sat_state_plot[:,7:11]  = sat_quat_targ

    return sat_state_plot

def get_3d_frame_data(plot_states, idx, scale):
    # Unpack input data
    pos         = plot_states[idx, 0:3]
    orient      = plot_states[idx, 3:7]
    quat_targ   = plot_states[idx, 7:11]

    # Create the spacecraft trajectory
    spacecraft_traj = draw_traj(plot_states[:, 0:3], idx, "Oranges")

    # Plot the spacecraft axes
    sc_axes    = draw_spacecraft_axes(pos, orient, scale)
    
    # Plot the target axes
    targ_axes   = draw_target_axes(pos, quat_targ, scale)

    # Add all the objects to the list
    objs = spacecraft_traj
    objs.extend(targ_axes)
    objs.extend(sc_axes)
    return objs

def create_3d_animation(fig, plotformat, sat_state_plot, update_rate):
    # Unpack the plot format
    title       = plotformat[0]
    scale       = plotformat[1]
    xaxis       = plotformat[2]
    yaxis       = plotformat[3]
    zaxis       = plotformat[4]

    # Get the data for each frame
    fig.update(frames=[go.Frame(data=get_3d_frame_data(sat_state_plot, i*update_rate, scale), 
                                traces=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]) 
                                for i in tqdm(range(1, round(len(sat_state_plot)/update_rate)))])
    
    fig.update_layout(title=dict(text=title),
                      font=dict(family="Courier New, monospace", color="RebeccaPurple"),
                      transition = {'duration': 1},
                      scene=dict(aspectmode="cube",
                      xaxis=xaxis, 
                      yaxis=yaxis, 
                      zaxis=zaxis),
                                 updatemenus=[dict(type="buttons",buttons=[dict(label="Play",method="animate",
                                                args=[None, {"frame": {"duration": 1, "redraw": True}}])])],
                        showlegend=False)
    return fig

def create_inertial_animimation(sat_pos, sat_orient, sat_targ_orient, update_rate):
    max_range = np.max(np.abs(sat_pos))

    min_axis = -1.2*max_range
    max_axis = 1.2*max_range
    scale   = max_axis / 7
    xaxis   = dict(range=[min_axis, max_axis], title=dict(text="X [m]"))
    yaxis   = dict(range=[min_axis, max_axis], title=dict(text="Y [m]"))
    zaxis   = dict(range=[min_axis, max_axis], title=dict(text="Z [m]"))

    plotformat = [f"ECI Trajectory", scale, xaxis, yaxis, zaxis]

    sat_state_plot  = states_for_plot(sat_pos, sat_orient, sat_targ_orient)

    frame_data  = get_3d_frame_data(sat_state_plot, 0, scale)
    frame_data.append(draw_planet(r_moon, "Greys"))
    fig         = go.Figure(data=frame_data)

    animation = create_3d_animation(fig, plotformat, sat_state_plot, update_rate)
    return animation

In [ ]:
animation = create_inertial_animimation(state_hist[:,0:3], state_hist[:,6:10], q_inertial_to_target_hist, len(times)//350)
animation.show()

In [ ]:
end_idx = idx_events[-1] + 100
times_ascent = times[0:end_idx]
state_hist_ascent = state_hist[0:end_idx]
q_inertial_to_target_hist_ascent = q_inertial_to_target_hist[0:end_idx]
LVCS_state_hist_ascent = LVCS_state_hist[0:end_idx]
thrust_dir_hist_ascent = thrust_dir_hist[0:end_idx]

fig = go.Figure(data=[go.Scatter(x=times_ascent, y=np.linalg.norm(state_hist_ascent[:,0:3], axis=1)-r_moon, mode="lines", name="Altitude [m]")])
fig.add_vline(x=times_ascent[idx_events[0]], line_dash="solid", line_color="red", annotation_text=f"<b>{event_names[0]}</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[2]], line_dash="dot", line_color="purple", annotation_text=f"<b>{event_names[2]}</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[3]], line_dash="dash", line_color="orange", annotation_text=f"<b>{event_names[3]}</b>", annotation_position="bottom left", annotation_font=dict(size=14))
fig.add_hline(y=R_D-r_moon, line_dash="dash", line_color="green", annotation_text="<b>Target Injection Altitude</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.update_layout(title="<b>Spacecraft Altitude</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Altitude [m]</b>", title_font=dict(size=14))
fig.show()

fig = go.Figure(data=[go.Scatter(x=times_ascent, y=np.linalg.norm(state_hist_ascent[:,3:6], axis=1), mode="lines", name="Velocity [m/s]")])
fig.add_vline(x=times_ascent[idx_events[0]], line_dash="solid", line_color="red", annotation_text=f"<b>{event_names[0]}</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[2]], line_dash="dot", line_color="purple", annotation_text=f"<b>{event_names[2]}</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[3]], line_dash="dash", line_color="orange", annotation_text=f"<b>{event_names[3]}</b>", annotation_position="bottom left", annotation_font=dict(size=14))
fig.add_hline(y=np.linalg.norm([R_D_dot, Y_D_dot, Z_D_dot]), line_dash="dash", line_color="green", annotation_text="<b>Target Injection Velocity</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.update_layout(title="<b>Spacecraft Velocity</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Velocity [m/s]</b>", title_font=dict(size=14))
fig.show()

error_quat = get_error_quats(state_hist_ascent[:,6:10], q_inertial_to_target_hist_ascent)
fig = go.Figure(data=[go.Scatter(x=times_ascent, y=error_quat[:,1], mode="lines", name=r"$\delta q_x$")])
fig.add_trace(go.Scatter(x=times_ascent, y=error_quat[:,2], mode="lines", name=r"$\delta q_y$"))
fig.add_trace(go.Scatter(x=times_ascent, y=error_quat[:,3], mode="lines", name=r"$\delta q_z$"))
fig.add_vline(x=times_ascent[idx_events[1]], line_dash="dash", line_color="green", annotation_text=f"<b>{event_names[1]}</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[2]], line_dash="dot", line_color="purple", annotation_text=f"<b>{event_names[2]}</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times[idx_events[3]], line_dash="dash", line_color="orange", annotation_text=f"<b>{event_names[3]}</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Error Quaternion</b>", font=dict(size=16))
fig.update_xaxes(title= "<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Error Quaternion Value</b>", title_font=dict(size=14))
fig.show()

fig = go.Figure(data=[go.Scatter(x=times_ascent, y=state_hist_ascent[:,10], mode="lines", name=r"$\omega_x$")])
fig.add_trace(go.Scatter(x=times_ascent, y=state_hist_ascent[:,11], mode="lines", name=r"$\omega_y$"))
fig.add_trace(go.Scatter(x=times_ascent, y=state_hist_ascent[:,12], mode="lines", name=r"$\omega_z$"))
fig.add_vline(x=times_ascent[idx_events[1]], line_dash="dash", line_color="green", annotation_text=f"<b>{event_names[1]}</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[2]], line_dash="dot", line_color="purple", annotation_text=f"<b>{event_names[2]}</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[3]], line_dash="dash", line_color="orange", annotation_text=f"<b>{event_names[3]}</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Spacecraft Angular Velocity</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Angular Velocity [rad/s]</b>", title_font=dict(size=14))
fig.show()

fig = go.Figure(data=[go.Scatter(x=times_ascent, y=state_hist_ascent[:,13], mode="lines", name="Mass [kg]")])
fig.add_vline(x=times_ascent[idx_events[0]], line_dash="solid", line_color="red", annotation_text=f"<b>{event_names[0]}</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[3]], line_dash="dash", line_color="orange", annotation_text=f"<b>{event_names[3]}</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Spacecraft Mass</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Mass [kg]</b>", title_font=dict(size=14))
fig.show()

fig = go.Figure(data=[go.Scatter(x=times_ascent, y=state_hist_ascent[:,13]-mass_dry, mode="lines", name="Mass [kg]")])
fig.add_vline(x=times_ascent[idx_events[0]], line_dash="solid", line_color="red", annotation_text=f"<b>{event_names[0]}</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[3]], line_dash="dash", line_color="orange", annotation_text=f"<b>{event_names[3]}</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Fuel Mass</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Mass [kg]</b>", title_font=dict(size=14))
fig.show()

fig = go.Figure(data=[go.Scatter(x=times_ascent, y=thrust_dir_hist_ascent[:,0], mode="lines", name=r"$a_\text{Radial}$")])
fig.add_trace(go.Scatter(x=times_ascent, y=thrust_dir_hist_ascent[:,1], mode="lines", name=r"$a_\text{Cross-Range}$"))
fig.add_trace(go.Scatter(x=times_ascent, y=thrust_dir_hist_ascent[:,2], mode="lines", name=r"$a_\text{Down-Range}$"))
fig.add_vline(x=times_ascent[idx_events[0]], line_dash="solid", line_color="red", annotation_text=f"<b>{event_names[0]}</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[2]], line_dash="dot", line_color="purple", annotation_text=f"<b>{event_names[2]}</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[3]], line_dash="dash", line_color="orange", annotation_text=f"<b>{event_names[3]}</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Thrust Direction in LVCS</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Unit Thrust Vector Components</b>", title_font=dict(size=14))
fig.show()

fig = go.Figure(data=[go.Scatter(x=times_ascent, y=(LVCS_state_hist_ascent[:,0]-R_D)/1000, mode="lines", name="Radial")])
fig.add_trace(go.Scatter(x=times_ascent, y=(LVCS_state_hist_ascent[:,1]-Y_D)/1000, mode="lines", name="Cross-Range"))
fig.add_vline(x=times_ascent[idx_events[0]], line_dash="solid", line_color="red", annotation_text=f"<b>{event_names[0]}</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[2]], line_dash="dot", line_color="purple", annotation_text=f"<b>{event_names[2]}</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[3]], line_dash="dash", line_color="orange", annotation_text=f"<b>{event_names[3]}</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Spacecraft LVCS State Error</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>LVCS State Error [km]</b>", title_font=dict(size=14))
fig.show()

fig = go.Figure(data=[go.Scatter(x=times_ascent, y=LVCS_state_hist_ascent[:,2]-R_D_dot, mode="lines", name=r"$V_\text{Radial}$")])
fig.add_trace(go.Scatter(x=times_ascent, y=LVCS_state_hist_ascent[:,3]-Y_D_dot, mode="lines", name=r"$V_\text{Cross-Range}$"))
fig.add_trace(go.Scatter(x=times_ascent, y=LVCS_state_hist_ascent[:,4]-Z_D_dot, mode="lines", name=r"$V_\text{Down-Range}$"))
fig.add_vline(x=times_ascent[idx_events[0]], line_dash="solid", line_color="red", annotation_text=f"<b>{event_names[0]}</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[2]], line_dash="dot", line_color="purple", annotation_text=f"<b>{event_names[2]}</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_ascent[idx_events[3]], line_dash="dash", line_color="orange", annotation_text=f"<b>{event_names[3]}</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Spacecraft LVCS Velocity Error</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>LVCS Velocity Error [m/s]</b>", title_font=dict(size=14))
fig.show()


In [ ]:
def create_alt_vs_down_range(time_hist, state_hist, thrust_dir_hist, idx_events, event_names, traj_points, end_idx=-1):
    if end_idx == -1:
        end_idx = len(time_hist)
    states  = state_hist[0:end_idx, :]
    times   = time_hist[0:end_idx]
    alts    = (np.linalg.norm(states[:,0:3], axis=1) - r_moon) / 1000
    thetas  = np.zeros(len(alts))
    down_ranges = np.zeros(len(alts))

    n = unit(np.cross(states[0,0:3], states[-1,0:3]))

    for i in range(1, len(alts)):
        in_plane = states[i, 0:3] - n*(np.dot(n, states[i, 0:3]))
        thetas[i] = np.arccos(np.dot(in_plane, states[0, 0:3])/(np.linalg.norm(in_plane)*np.linalg.norm(states[0,0:3])))
        down_ranges[i] = (r_moon * thetas[i]) / 1000
        if np.isnan(down_ranges[i]):
            down_ranges[i] = 0

    arrow_angle = np.rad2deg(np.arcsin(thrust_dir_hist[:,2]))

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=down_ranges, y=alts, mode='lines', name="Trajectory", showlegend=False))
    fig.add_trace(go.Scatter(x=down_ranges[traj_points], y=alts[traj_points], mode='markers', name=f"Position along Trajectory", marker=dict(symbol="arrow-wide", size=12, angle=arrow_angle[traj_points]), showlegend=False))

    if len(idx_events) > 0 and idx_events[0] < end_idx:
        t_lo = times - times[idx_events[0]]
        min_event = (t_lo // 60).astype(int)
        sec_event = t_lo % 60
        fig.add_trace(go.Scatter(x=[down_ranges[idx_events[0]]], y=[alts[idx_events[0]]], mode="markers", marker=dict(symbol="star-triangle-up", size=20), name=f"<b>{event_names[0]}</b><br>T+{min_event[idx_events[0]]:02d}:{sec_event[idx_events[0]]:05.2f}<br>Alt: {alts[idx_events[0]]*1000:.2f}m<br>Downrange: {down_ranges[idx_events[0]]:.2f} km"))
        
    if len(idx_events) > 1 and idx_events[1] < end_idx:
        fig.add_trace(go.Scatter(x=[down_ranges[idx_events[1]]], y=[alts[idx_events[1]]], mode='markers', marker=dict(symbol="diamond-wide", size=17), name=f"<b>{event_names[1]}</b><br>T+{min_event[idx_events[1]]:02d}:{sec_event[idx_events[1]]:05.2f}<br>Alt: {alts[idx_events[1]]*1000:.2f}m<br>Downrange: {down_ranges[idx_events[1]]:.2f} km"))

        if len(idx_events) > 2 and idx_events[2] < end_idx:
            fig.add_trace(go.Scatter(x=[down_ranges[idx_events[2]]], y=[alts[idx_events[2]]], mode='markers', marker=dict(symbol="diamond-tall", size=15), name=f"<b>{event_names[2]}</b><br>T+{min_event[idx_events[2]]:02d}:{sec_event[idx_events[2]]:05.2f}<br>Alt: {alts[idx_events[2]]*1000:.2f}m<br>Downrange: {down_ranges[idx_events[2]]:.2f} km"))

            if len(idx_events) > 3 and idx_events[3] < end_idx:
                fig.add_trace(go.Scatter(x=[down_ranges[idx_events[3]]], y=[alts[idx_events[3]]], mode='markers', marker=dict(symbol="octagon", size=20), name=f"<b>{event_names[3]}</b><br>T+{min_event[idx_events[3]]:02d}:{sec_event[idx_events[3]]:05.2f}<br>Alt: {alts[idx_events[3]]:.2f}km<br>Downrange: {down_ranges[idx_events[3]]:.2f} km"))
    fig.update_layout(title='<b>Altitude vs. Downrange Distance</b>', font=dict(size=16))
    fig.update_xaxes(title='<b>Downrange Distance [km]</b>', title_font=dict(size=14))
    fig.update_yaxes(title='<b>Altitude [km]</b>', title_font=dict(size=14))
    fig.update_layout(legend=dict(font=dict(size=11)))
    return fig

In [ ]:
end_idx_plt = 175
traj_points = np.arange(0, end_idx_plt, 10)
fig = create_alt_vs_down_range(times_ascent, state_hist_ascent, thrust_dir_hist_ascent, idx_events, event_names, traj_points, end_idx=end_idx_plt)
fig.show()

traj_points = np.arange(0, len(times_ascent), 250)
fig = create_alt_vs_down_range(times_ascent, state_hist_ascent, thrust_dir_hist_ascent, idx_events, event_names, traj_points)
fig.show()

In [ ]:
print("LVCS State Error at Cut-Off:")
print(f"Radial:\t\t\t{(LVCS_state_hist_ascent[idx_events[3],0]-R_D):.3f} m")
print(f"Cross-Range:\t\t{(LVCS_state_hist_ascent[idx_events[3],1]-Y_D):.3f} m")
print(f"Radial Velocity:\t{(LVCS_state_hist_ascent[idx_events[3],2]-R_D_dot):.3f} m/s")
print(f"Cross-Range Velocity:\t{(LVCS_state_hist_ascent[idx_events[3],3]-Y_D_dot):.3f} m/s")
print(f"Downrange Velocity:\t{(LVCS_state_hist_ascent[idx_events[3],4]-Z_D_dot):.3f} m/s")